Starting with the file most recent file (2013 - September 10, 2018). Working with Bainbridge departures only.

In [162]:
import pandas as pd
import numpy as np

laptop_file_path = 'C:/Users/Keith/Google Drive/H Drive Full Backup/Fall 2018/CS 495 - Senior Seminar/Ferry Data/P000413-090518_2013-2018-09-10.csv'
dataset = pd.read_csv(laptop_file_path)

bainbridge_departures = dataset.loc[dataset['departure_terminal'] == 'Bainbridge'].reset_index()
bainbridge_departures = bainbridge_departures.drop('index', axis=1)

In [163]:
bainbridge_departures.head()

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime
0,Bainbridge,Colman,55:00.0,05:56.0,37:37.0,32.0
1,Bainbridge,NaN,45:00.0,NaN,NaN,NaN
2,Bainbridge,Colman,20:00.0,19:01.0,49:40.0,30.0
3,Bainbridge,NaN,20:00.0,NaN,NaN,NaN
4,Bainbridge,NaN,05:00.0,06:16.0,NaN,NaN


<a href = "https://web.archive.org/web/20130123233819/http:/www.wsdot.wa.gov/ferries/pdf/2013winter.pdf"> Winter 2013 Schedule </a>

In [164]:
# Monday-Friday leave Bainbridge Island Winter 2013
# mfw13 means Monday-Friday Winter 2013
mfw13_scheduled_departures_24h = ["0:55:00", "4:45:00", "5:20:00", "6:20:00", "7:05:00", "7:55:00","8:45:00", "9:40:00", 
                            "10:25:00", "11:30:00", "12:20:00", "13:10:00", "14:05:00", "14:55:00", "15:50:00", "16:35:00", 
                            "17:30:00", "18:30:00", "19:10:00", "20:10:00", "20:55:00", "21:45:00", "22:35:00"]

# satw13 means Saturday Winter 2013
satw13_scheduled_departures_24h = ["0:55:00","05:20:00","07:05:00","07:55:00","08:45:00","9:35:00", "10:25:00",
                                  "11:30:00","12:20:00","13:10:00","14:05:00","14:55:00","15:50:00",
                                  "16:35:00","17:30:00","18:30:00","19:10:00","20:10:00","20:55:00",
                                  "21:45:00","22:30:00","0:00:00","1:25:00"]

# sunw13 means Sunday Winter 2013
sunw13_scheduled_departures_24h = ["05:20:00","07:05:00","08:45:00","9:35:00", "10:25:00",
                                  "11:30:00","12:20:00","13:10:00","14:05:00","14:55:00","15:50:00",
                                  "16:35:00","17:30:00","18:30:00","19:10:00","20:10:00","20:55:00",
                                  "21:45:00","22:30:00","0:00:00","1:25:00"]

#wkw13 means Week Winder 2013
wkw13_scheduled_departures_24h = (5*mfw13_scheduled_departures_24h) + satw13_scheduled_departures_24h + sunw13_scheduled_departures_24h

In [165]:
def extractMinutesPastHour(time):
    m = time.split(':')
    return int(float(m[0]))

# For time in format hh:mm:ss this method returns mm
def extractMinutes24h(time):
    m = time.split(':')
    return int(m[1])

In [166]:
import datetime

def generate_timestamps(dataset, scheduled_departures_24h, starting_point, year, month, day):
    #import pdb; pdb.set_trace()

    tdelta = datetime.timedelta(days = 1)

    prevHour = 0

    departure_times = []
    timestamps = []
    weekdays = []

    i = starting_point

    for index, row in dataset.iterrows():
        departure24h = scheduled_departures_24h[i % len(scheduled_departures_24h)]
        hour, minute, seconds = departure24h.split(":")
        scheduled_departure_minutes = extractMinutesPastHour(row.iloc[2])
        scheduled_departure_24h = extractMinutes24h(departure24h)

        if(int(prevHour) > int(hour)):
            d = d + tdelta
            year = d.year
            month = d.month
            day = d.day

        if (not scheduled_departure_minutes == scheduled_departure_24h):
            if(d.weekday() == 1):
                # --- this is a quick fix not sure why this works ----
                d = d - tdelta
                year = d.year
                month = d.month
                day = d.day
                # ------------------------------------

                i = i + 1

                departure24h = scheduled_departures_24h[i % len(scheduled_departures_24h)]
                scheduled_hour, scheduled_minutes, scheduled_seconds = departure24h.split(":")
                scheduled_departure_minutes = extractMinutesPastHour(row.iloc[2])

                departure_times.append(departure24h)
                d = datetime.date(year, month, day)
                timestamps.append("%s %02d:%02d:%02d" % (d, int(scheduled_hour), int(scheduled_minutes), int(scheduled_seconds)))
                weekdays.append(d.weekday())

                prevHour = hour  
                i = i + 1
                continue
                if (not scheduled_departure_minutes == scheduled_minutes):
                    print("ERROR")
                    break

            print("""Was able to match departure times up until this row at index %d:\n%s\n
                    Was trying to insert this timestamp: %s""" % 
                                              (index, row,("%s %02d:%02d:%02d" % (d, int(hour), int(minute), int(seconds)))))
            break

        departure_times.append(departure24h)
        d = datetime.date(year, month, day)
        timestamps.append("%s %02d:%02d:%02d" % (d, int(hour), int(minute), int(seconds)))
        weekdays.append(d.weekday())

        prevHour = hour
        i = i + 1
        
    return timestamps

In [181]:
timestamps = generate_timestamps(bainbridge_departures, wkw13_scheduled_departures_24h, 23, 2013, 1, 1)

Was able to match departure times up until this row at index 3297:
departure_terminal     Bainbridge
arrival_terminal           Colman
scheduled_departure       05:00.0
actual_departure          05:30.0
Actual_Arrival            36:59.0
TransitTime                    31
Name: 3297, dtype: object

                    Was trying to insert this timestamp: 2013-05-27 05:20:00


In [182]:
len(timestamps)

3297

In [183]:
timestamps[3290:]

['2013-05-26 20:10:00',
 '2013-05-26 20:55:00',
 '2013-05-26 21:45:00',
 '2013-05-26 22:30:00',
 '2013-05-27 00:00:00',
 '2013-05-27 01:25:00',
 '2013-05-27 04:45:00']

Want to git rid of those last three dates.

In [184]:
timestamps = timestamps[:3296]

In [185]:
len(timestamps)

3296

May 27, 2013 Starts at index 3296 in `bainbridge_departures`

In [189]:
bainbridge_departures.iloc[3296:3300]

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime
3296,Bainbridge,Colman,20:00.0,20:11.0,51:37.0,31.0
3297,Bainbridge,Colman,05:00.0,05:30.0,36:59.0,31.0
3298,Bainbridge,Colman,55:00.0,56:22.0,26:55.0,30.0
3299,Bainbridge,Colman,45:00.0,45:00.0,16:17.0,31.0


The only difference between the <a href = "https://web.archive.org/web/20130123233819/http:/www.wsdot.wa.gov/ferries/pdf/2013winter.pdf"> Winter 2013 Schedule</a> and the <a href = "https://web.archive.org/web/20130624071202/http:/www.wsdot.wa.gov/ferries/pdf/2013Spring.pdf"> Spring 2013 Schedule</a> happend on May 27, 2013

In [152]:
# May 27, 2013
may2713 = ["05:20:00","07:05:00","07:55:00","08:45:00","9:35:00", "10:25:00","11:30:00","12:20:00","13:10:00",
           "14:05:00","14:55:00","15:50:00","16:35:00","17:30:00","18:30:00","19:10:00","20:10:00","20:55:00",
           "21:45:00","22:30:00","0:00:00","1:25:00"]

In [190]:
may2713timestamps = generate_timestamps(bainbridge_departures.iloc[3296:], may2713, 0, 2013, 5, 27)

Was able to match departure times up until this row at index 3319:
departure_terminal     Bainbridge
arrival_terminal           Colman
scheduled_departure       20:00.0
actual_departure          20:00.0
Actual_Arrival            51:12.0
TransitTime                    31
Name: 3319, dtype: object

                    Was trying to insert this timestamp: 2013-05-27 07:55:00


In [193]:
may2713timestamps = may2713timestamps[:22]

In [195]:
timestamps = timestamps + may2713timestamps

This below is the beginning of Tuesday, May 28, 2013

In [202]:
bainbridge_departures.iloc[3318:3322]

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime
3318,Bainbridge,Colman,45:00.0,45:56.0,16:44.0,31.0
3319,Bainbridge,Colman,20:00.0,20:00.0,51:12.0,31.0
3320,Bainbridge,Colman,20:00.0,23:59.0,51:35.0,28.0
3321,Bainbridge,Colman,05:00.0,06:51.0,40:22.0,34.0


In [204]:
len(mfw13_scheduled_departures_24h)

23

In [207]:
timestamps_2 = generate_timestamps(bainbridge_departures.iloc[3318:], wkw13_scheduled_departures_24h, 24, 2013, 5, 28)

Was able to match departure times up until this row at index 4154:
departure_terminal     Bainbridge
arrival_terminal           Colman
scheduled_departure       20:00.0
actual_departure          19:33.0
Actual_Arrival            49:12.0
TransitTime                    30
Name: 4154, dtype: object

                    Was trying to insert this timestamp: 2013-07-04 04:45:00


In [208]:
timestamps = timestamps + timestamps_2

In [209]:
len(timestamps)

4154

In [217]:
timestamps[4150:]

['2013-07-03 20:55:00',
 '2013-07-03 21:45:00',
 '2013-07-03 22:35:00',
 '2013-07-04 00:55:00']

The fourth of July schedule is the same as the May 27, 2013 schedule.

Thursday, July 4, 2013

In [220]:
july0413timestamps = generate_timestamps(bainbridge_departures[4154:], may2713, 0, 2013, 7, 4)

Was able to match departure times up until this row at index 4176:
departure_terminal     Bainbridge
arrival_terminal           Colman
scheduled_departure       45:00.0
actual_departure          44:36.0
Actual_Arrival            13:56.0
TransitTime                    29
Name: 4176, dtype: object

                    Was trying to insert this timestamp: 2013-07-05 05:20:00


In [224]:
timestamps = timestamps + july0413timestamps

In [225]:
len(timestamps)

4176

Need to start again this time from Thursday

In [234]:
timestamps_2 = generate_timestamps(bainbridge_departures[4176:], wkw13_scheduled_departures_24h, (23*4) + 1, 2013, 7, 5)

Was able to match departure times up until this row at index 5507:
departure_terminal     Bainbridge
arrival_terminal           Colman
scheduled_departure       05:00.0
actual_departure          05:06.0
Actual_Arrival            32:58.0
TransitTime                    27
Name: 5507, dtype: object

                    Was trying to insert this timestamp: 2013-09-02 05:20:00


In [235]:
timestamps = timestamps + timestamps_2

Monday, September 2, 2013 was a holiday.

In [242]:
timestamps = timestamps[:5506]

In [243]:
len(timestamps)

5506

In [244]:
timestamps[5500:]

['2013-09-01 20:10:00',
 '2013-09-01 20:55:00',
 '2013-09-01 21:45:00',
 '2013-09-01 22:30:00',
 '2013-09-02 00:00:00',
 '2013-09-02 01:25:00']

In [246]:
bainbridge_departures.iloc[5506:].head()

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime
5506,Bainbridge,Colman,20:00.0,20:16.0,49:50.0,29.0
5507,Bainbridge,Colman,05:00.0,05:06.0,32:58.0,27.0
5508,Bainbridge,Colman,55:00.0,55:00.0,26:35.0,31.0
5509,Bainbridge,Colman,45:00.0,45:06.0,14:50.0,29.0
5510,Bainbridge,Colman,35:00.0,35:36.0,06:01.0,31.0


In [247]:
sept0213timestamps = generate_timestamps(bainbridge_departures[5506:], may2713, 0, 2013, 9, 2)

Was able to match departure times up until this row at index 5529:
departure_terminal     Bainbridge
arrival_terminal           Colman
scheduled_departure       20:00.0
actual_departure          20:05.0
Actual_Arrival            53:03.0
TransitTime                    33
Name: 5529, dtype: object

                    Was trying to insert this timestamp: 2013-09-02 07:55:00


In [250]:
sept0213timestamps = sept0213timestamps[:22]

In [252]:
timestamps = timestamps + sept0213timestamps

In [253]:
len(timestamps)

5528

In [254]:
timestamps[5520:]

['2013-09-02 18:30:00',
 '2013-09-02 19:10:00',
 '2013-09-02 20:10:00',
 '2013-09-02 20:55:00',
 '2013-09-02 21:45:00',
 '2013-09-02 22:30:00',
 '2013-09-03 00:00:00',
 '2013-09-03 01:25:00']

In [256]:
bainbridge_departures.iloc[5528:].head()

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime
5528,Bainbridge,Colman,45:00.0,45:07.0,12:15.0,27.0
5529,Bainbridge,Colman,20:00.0,20:05.0,53:03.0,33.0
5530,Bainbridge,Colman,20:00.0,20:00.0,49:57.0,29.0
5531,Bainbridge,Colman,05:00.0,06:03.0,36:39.0,30.0
5532,Bainbridge,Colman,55:00.0,56:43.0,26:53.0,30.0


Tuesday, September 3, 2013 (is the start of whats above)